# Introduction by example

## Data handling of graphs

In [1]:
import torch
from torch_geometric.data import Data

In [7]:
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
data = Data(x=x, edge_index=edge_index)

In [8]:
print(data.keys)

['x', 'edge_index']


In [9]:
print(data['x'])

tensor([[-1.],
        [ 0.],
        [ 1.]])


## Common benchmark datasets

In [12]:
from torch_geometric.datasets import TUDataset
dataset = TUDataset(root='./data/ENZYMES', name='ENZYMES')

Extracting data/ENZYMES/ENZYMES.zip
Processing...
Done!


In [14]:
print(len(dataset))
print(dataset.num_classes)
print(dataset.num_features)

600
6
3


In [33]:
data = dataset[0]
print(data)
print(data.is_undirected)

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])
<bound method Data.is_undirected of Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])>


In [18]:
train_dataset = dataset[:540]
test_dataset = dataset[540:]
print(train_dataset, test_dataset)

ENZYMES(540) ENZYMES(60)


In [19]:
from torch_geometric.datasets import Planetoid

In [21]:
dataset = Planetoid(root='./data/Cora', name='Cora')
print(len(dataset))
print(dataset.num_classes)
print(dataset.num_features)

1
7
1433


In [31]:
data = dataset[0]
print(data)

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])


In [29]:
print(data.is_undirected())
print(data.train_mask.sum())
print(data.val_mask.sum())
print(data.test_mask.sum())

True
tensor(140)
tensor(500)
tensor(1000)


## Mini-batches

## Data transforms


## Learning methods on graphs

In [34]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='./data/Cora', name='Cora')

In [123]:
### import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__()
        # hidden_channels = int((in_channels + out_channels) / 2)
        self.fc1 = torch.nn.Linear(in_channels, out_channels)
        # self.fc2 = torch.nn.Linear(hidden_channels, out_channels)
        
    def forward(self, x, edge_index):
        print('1 ', x.shape)
        print('2 ', edge_index.shape)
        edge_index = add_self_loops(edge_index, num_nodes=x.size(0))
        print('3 ', edge_index.shape)
        x = self.fc1(x)
        print('4 ', x.shape)
        # x = self.fc2(x)
        
        return self.propagate('add', edge_index, x=x, num_nodes=x.size(0))
    
    def message(self, x_j, edge_index, num_nodes):
        print(x_j.shape)
        print(x_j)
        row, col = edge_index
        deg = degree(row, num_nodes, dtype=x_j.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        
        return norm.view(-1, 1) * x_j
    
    def update(self, aggr_out):
        return aggr_out

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)

In [124]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [125]:
model.train()
for epoch in range(1):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

1  torch.Size([2708, 1433])
2  torch.Size([2, 10556])
3  torch.Size([2, 13264])
4  torch.Size([2708, 16])
torch.Size([13264, 16])
tensor([[-0.0508, -0.0588, -0.0723,  ..., -0.0734, -0.0315,  0.0635],
        [-0.0990, -0.0931, -0.0245,  ..., -0.0426,  0.0052, -0.0444],
        [-0.0989, -0.1004,  0.0133,  ..., -0.0405,  0.1329, -0.0556],
        ...,
        [-0.0081, -0.0177, -0.0176,  ...,  0.0519,  0.0237,  0.0638],
        [-0.0796, -0.0813, -0.0062,  ...,  0.0075, -0.0529, -0.0666],
        [-0.0888, -0.0513,  0.0584,  ...,  0.0244,  0.0416, -0.0452]],
       grad_fn=<TakeBackward>)
1  torch.Size([2708, 16])
2  torch.Size([2, 10556])
3  torch.Size([2, 13264])
4  torch.Size([2708, 7])
torch.Size([13264, 7])
tensor([[ 0.1680, -0.0413,  0.1184,  ...,  0.0178, -0.1576, -0.1840],
        [ 0.1974, -0.0442,  0.1758,  ..., -0.0260, -0.1730, -0.1369],
        [ 0.1838, -0.0679,  0.1482,  ..., -0.0090, -0.1749, -0.1676],
        ...,
        [ 0.2045, -0.1204,  0.1175,  ..., -0.0695, -0.19

In [119]:
model.eval()
_, pred = model(data).max(dim=1)
print(pred)
correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

torch.Size([13264, 16])
torch.Size([13264, 7])
tensor([6, 4, 4,  ..., 6, 6, 6])
Accuracy: 0.2430
